# H.9 Qubitization
Show Textbook
The content in this node is advanced. We recommend reading the sections on Grover search and quantum phase estimation beforehand.

The  and  oracles can be used for a different method of Hamiltonian simulation called qubitization. The basic observation is that, when the component unitaries  in a Hamiltonian

Author: [Monit Sharma](https://github.com/MonitSharma)
LinkedIn: [Monit Sharma](https://www.linkedin.com/in/monitsharma/)
Twitter: [@MonitSharma1729](https://twitter.com/MonitSharma1729)
Medium : [MonitSharma](https://medium.com/@_monitsharma)

![](https://codebook.xanadu.ai/pics/qubitization_alt_1.svg)

### Codercise H.9.1.
 (a) Let's start by implementing the walk operator . The functions SELECT(U_list) and PREPARE_matrix(alpha_list) are available to you. Recall that PREPARE_matrix produces a matrix, while SELECT is a subcircuit, applying a sequence of gates. Note: you'll need to make the  matrix and use qml.QubitUnitary to implement it the circuit level.

In [ ]:
k_bits = 2 # number of bits in the O register
main_bits = 2 # number of bits in the main register
all_bits = k_bits + main_bits # total number of bits

# define wire ranges for each of the three registers
k_wires = range(k_bits) # O register
main_wires = range(k_bits, all_bits) # main register
dev = qml.device("default.qubit", wires=all_bits)  

def walk(alpha_list, U_list):
    """Create a subcircuit for the walk operator in qubitization.
    
    Args:
        alpha_list (array[float]): A list of coefficients.
        U_list (list[array[complex]]): A list of unitary matrices, stored as complex arrays.
    """

    ##################
    # YOUR CODE HERE #
    ##################
    SELECT(U_list)
    mat = PREPARE_matrix(alpha_list)
    r = np.zeros(shape=mat.shape)
    for _ in range(len(r)):
        if _ == 0:
            r[_][_] = 1.0
            continue
        r[_][_] = -1.0
    R = mat.conj().T @ r @ mat
    qml.QubitUnitary(R, wires=k_wires)



(b) Here is a circuit with the initialization steps and a single application of  shown:

![](https://codebook.xanadu.ai/pics/qubitization_codercise_2.svg)

In [ ]:
@qml.qnode(dev)
def walk_circuit(alpha_list, U_list, steps):
    """Create a subcircuit for the walk operator in qubitization.
    
    Args:
        alpha_list (array[float]): A list of coefficients.
        U_list (list[array[complex]]): A list of unitary matrices, stored as complex arrays.
        steps (int): The number of times to iterate the walk operator.

    Returns:
        array[complex]: The quantum state after applying the circuit.
    """

    ##################
    # YOUR CODE HERE #
    ##################
    qml.QubitUnitary(PREPARE_matrix(alpha_list), wires=k_wires)
    for _ in range(steps):
        walk(alpha_list,U_list)
        

    return qml.state()


### Codercise H.9.2. 
(a) Write a circuit which prepares the  eigenstate in Eq. (). Specifically, your circuit must apply the  operator,

In [ ]:
targ_bits = k_bits + main_bits
targ_wires = range(targ_bits)
k_wires = range(k_bits)

def eigenstate_prep(alpha_list, U_list, E):
    """Create a subcircuit which prepares the + eigenstate of the walk operator.
    
    Args:
        alpha_list (array[float]): A list of coefficients.
        U_list (list[array[complex]]): A list of unitary matrices.
        E (float): Our guess at the energy of the state.
    """
    alpha = sum(alpha_list)

    ##################
    # YOUR CODE HERE #
    ##################

    # DEFINE c_+/- 
    c_plus =  np.sqrt(2 * (1 + E/alpha))**(-1)
    
    c_minus = np.sqrt(2 * (1 - E/alpha))**(-1)
    qml.QubitUnitary(PREPARE_matrix(alpha_list), wires= k_wires)
    mat = SELECT_matrix(U_list)
    mat = 1/np.sqrt(2) * ((c_plus + 1j * c_minus) * np.eye(2**targ_bits) +  (c_plus - 1j * c_minus) * mat)
    
    qml.QubitUnitary(mat, wires=targ_wires)
    c = [c_plus, c_minus] 
    


![](https://codebook.xanadu.ai/pics/qubitization_alt_6.svg)

b) Complete the following code for estimating the phase. Since the qml.QuantumPhaseEstimation subroutine requires us to specify the unitary as a matrix, the walk operator is provided as a matrix for you via walk_matrix(alpha_list, U_list). The eigenstate_prep function is also available to use. We'll also allow for initialization of computational basis states on the main register. Note that the targ_bits, targ_wires, and k_bits variables from the previous exercise are available to you here, as well.

In [ ]:
p_bits = 8 # number of bits in the T register
# range of wires in the T register (where phase estimation occurs)
p_wires = range(targ_bits, targ_bits + p_bits) 

dev2 = qml.device("default.qubit", wires=targ_bits + p_bits)

@qml.qnode(dev2)
def qpe_circuit(alpha_list, U_list, state, E):
    """Create a circuit for estimating the phase of a basis state of the walk operator.
    
    Args:
        alpha_list (array[float]): A list of coefficients.
        U_list (list[array[complex]]): A list of unitary matrices.
        state (list[int]): A basis state, specified as a list of bits.
        E (float): Our guess at the energy of the state.

    Returns:
        array[float]: The probabilities on the estimate register.
    """

    ##################
    # YOUR CODE HERE #
    ##################

    # INITIALIZE BASIS STATE
    # PREPARE EIGENSTATE OF WALK OPERATOR
    eigenstate_prep(alpha_list,U_list,E)

    # APPLY QUANTUM PHASE ESTIMATION
    qml.QuantumPhaseEstimation(walk_matrix(alpha_list,U_list),targ_wires,p_wires)

    return qml.probs(wires=p_wires)


(c) Confirm that your phase estimation circuit qpe_circuit works by applying it to the simple Hamiltonian above. Populate the U_list and alpha_list lists so that you can calculate two phases: phase_estimate_00 and phase_estimated_01. With these phases, the code provided will extract the energies!

In [ ]:
U_list = [np.kron(qml.Hadamard.compute_matrix(), qml.Hadamard.compute_matrix()),
        np.kron(-qml.Hadamard.compute_matrix(), qml.PauliX.compute_matrix()),
        np.kron(-qml.PauliX.compute_matrix(), qml.Hadamard.compute_matrix()),
        np.kron(qml.PauliX.compute_matrix(), qml.PauliX.compute_matrix())] # MODIFY THIS
alpha_list = [2.0, np.sqrt(2), np.sqrt(2),1.0] # MODIFY THIS

phase_estimated_00 = qpe_circuit(alpha_list, U_list,[0,0],1.0) # MODIFY THIS
phase_estimated_01 = qpe_circuit(alpha_list, U_list,[0,1],-1.0) # MODIFY THIS

phase_estimated_00 = phase_estimated = np.argmax(phase_estimated_00) / 2 ** p_bits
phase_estimated_01 = phase_estimated = np.argmax(phase_estimated_01) / 2 ** p_bits

alpha = sum(alpha_list)

energy_estimated_00 = alpha*np.cos(2*np.pi*phase_estimated_00)
energy_estimated_01 = alpha*np.cos(2*np.pi*phase_estimated_01)

print("The estimated energy for state |00> is", energy_estimated_00, ".")
print("The estimated energy for state |01> is", energy_estimated_01, ".")


### Codercise H.9.3.
 (a) Implement the matrix representation of the unitary . Note that it is diagonal, acts on both the target and estimation registers, and with matrix elements , where  is defined above and  runs discretely over all -decimal binary angles

In [ ]:
def S(time, alpha):
    """Implement the unitary S as a matrix.
    
    Args:
        time (float): The time to evolve the Hamiltonian for.
        alpha (float): The sum of alpha coefficients in the Hamiltonian.

    Returns:
        array[complex]: The matrix representation of S.
    """
    hbar = 1e-34
    output = np.eye(2**(targ_bits + p_bits), dtype = 'complex_')
    for k in range(2**targ_bits):
        for i in range(2**p_bits):
            index = 2**p_bits*k + i

            ##################
            # YOUR CODE HERE #
            ##################
            val = - time * alpha / hbar * np.cos(2 * np.pi * 1/(2**p_bits) * index)
            output[index, index] = np.exp(1j * val) # MODIFY THIS

    return output


![](https://codebook.xanadu.ai/pics/qubitization_alt_7.svg)

(b) Implement this simplified qubitization circuit. The  matrix function that you made in the previous exercise is callable with S(time, alpha). In this particular problem, we are going to simulate the following one-qubit Hamiltonian in the interest of getting a very accurate phase estimate:

In [ ]:
@qml.qnode(dev2)
def qubitization(alpha_list, U_list, time):
    """Perform Hamiltonian simulation using a simplified qubitization circuit.
    
    Args:
        alpha_list (array[float]): A list of coefficients.
        U_list (list[array[complex]]): A list of unitary matrices, stored as complex arrays.
        time (float): The time to evolve the Hamiltonian for.
    """
    prep = PREPARE_matrix(alpha_list)
    prep_dagger = np.conjugate(np.transpose(prep))
    alpha = sum(alpha_list)
    
    ##################
    # YOUR CODE HERE #
    ##################
    qml.QubitUnitary(prep,wires = [_ for _ in range(k_bits)])
    qml.QuantumPhaseEstimation(walk_matrix(alpha_list,U_list),targ_wires, p_wires)
    qml.QubitUnitary(S(time, alpha), wires = [_ for _ in range(10)])
    qml.adjoint(qml.QuantumPhaseEstimation)(walk_matrix(alpha_list,U_list),targ_wires,p_wires)
    qml.QubitUnitary(prep_dagger, wires = [_ for _ in range(k_bits)])
    
    return qml.probs(wires=range(k_bits, k_bits + main_bits))
